In [1]:
import pandas as pd
import math

# Random forest model training

In [2]:
## past annotated data in the first phase of TT project
past_data = pd.read_csv('./annotated_data.big_set.corrected.txt', sep=';' , encoding='latin-1', 
                        header = None, 
                        names = ['doc', 'col1', 'col2', 'text', 'label1', 'label2', 'label3'])

In [3]:
## annotated data by the German team
data = pd.read_excel('./evaluation_20220927.ods', engine = 'odf')

In [4]:
## identify table of content
data['inhalt1'] = data['text'].str.contains('Inhaltsverzeichnis', case=True).apply(lambda x: 1 if x else 0)
data['inhalt2'] = data['text'].str.contains('Inhalt', case=True).apply(lambda x: 1 if x else 0)
inhalt1_sum = data.groupby('file').sum('inhalt1')
inhalt1_ind = inhalt1_sum[inhalt1_sum['inhalt1'] > 0].index.tolist()
inhalt2_ind = set(inhalt1_sum.index) - set(inhalt1_ind)

data['exclude_inhalt'] = False



list_exclude = []
for i in inhalt1_ind:
    data_part = data[data['file'] == i]
    ind_start = data_part.index.tolist()[0]
    data_part = data_part[data_part['inhalt1'] == True]
    if len(data_part) > 0:
        ind_end = data_part[data_part['inhalt1'] == True].index[0]
        range_inhalt = list(range(ind_start, ind_end+1))
        if len(range_inhalt) <= 30: # change
            list_exclude = list_exclude + range_inhalt
            
for i in inhalt2_ind:
    data_part = data[data['file'] == i]
    ind_start = data_part.index.tolist()[0]
    data_part = data_part[data_part['inhalt2'] == True]
    if len(data_part) > 0:
        ind_end = data_part[data_part['inhalt2'] == True].index[0]
        range_inhalt = list(range(ind_start, ind_end+1))
        if len(range_inhalt) <= 30: # change
            list_exclude = list_exclude + range_inhalt
            
            
data['exclude_inhalt'][list_exclude] = True

/tmp/ipykernel_679427/2894172796.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['exclude_inhalt'][list_exclude] = True


In [5]:
# identify bibliography

data['inhalt1'] = data['text'].str.contains('Bibliographie|Bibliografie|Literaturverzeichnis', case=True).apply(lambda x: 1 if x else 0)
data['inhalt2'] = data['text'].str.contains('Literatur', case=True).apply(lambda x: 1 if x else 0)
inhalt1_sum = data.groupby('file').sum('inhalt1')
inhalt1_ind = inhalt1_sum[inhalt1_sum['inhalt1'] > 0].index.tolist()
inhalt2_ind = set(inhalt1_sum.index) - set(inhalt1_ind)
data['exclude_inhalt1'] = False

In [6]:
list_exclude = []
for i in inhalt1_ind:
    data_part = data[data['file'] == i]
    ind_start = data_part.index.tolist()[-1]
    data_part = data_part[data_part['inhalt1'] == True]
    if len(data_part) > 0:
        ind_end = data_part.index[-1]
        range_inhalt = list(range(ind_end, ind_start+1))
        if len(range_inhalt) <= 75: # change
            list_exclude = list_exclude + range_inhalt
            
for i in inhalt2_ind:
    data_part = data[data['file'] == i]
    ind_start = data_part.index.tolist()[-1]
    data_part = data_part[data_part['inhalt2'] == True]
    if len(data_part) > 0:
        ind_end = data_part.index[-1]
        range_inhalt = list(range(ind_end, ind_start+1))
        if len(range_inhalt) <= 75: # change
            list_exclude = list_exclude + range_inhalt

In [7]:
data['exclude_inhalt1'][list_exclude] = True

/tmp/ipykernel_679427/2453373711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['exclude_inhalt1'][list_exclude] = True


In [8]:
# calculate the number of token in each paragraph and remove those with too less tokens

data['text'] = data['text'].apply(lambda x: str(x))

In [9]:
data['num_token'] = data['text'].str.replace('.', '').str.replace(r"\s+", ' ').apply(lambda x: len(x.split(' ')))

/tmp/ipykernel_679427/2130238111.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['num_token'] = data['text'].str.replace('.', '').str.replace(r"\s+", ' ').apply(lambda x: len(x.split(' ')))
/tmp/ipykernel_679427/2130238111.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['num_token'] = data['text'].str.replace('.', '').str.replace(r"\s+", ' ').apply(lambda x: len(x.split(' ')))


In [10]:
data['exclude_num_token'] = (data['num_token'] < 25) # change

In [11]:
# remove all irrelevant paragraphs identifed by rule based methods above
data = data[data['exclude_inhalt'] == False][data['exclude_inhalt1'] == False][data['exclude_num_token'] == False]

/tmp/ipykernel_679427/695616058.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data[data['exclude_inhalt'] == False][data['exclude_inhalt1'] == False][data['exclude_num_token'] == False]


In [12]:
# Since the TT-I annotated data is at sentence level, merge them into paragraph level based on the average length of paragraphs in our current data
text = []
doc = []
for i in past_data['doc'].unique():
    df = past_data[past_data['doc'] == i]
    length = math.ceil(len(df)/5)
    for h in range(length):
        text.append(df[h*5:(h+1)*5]['text'].str.cat(sep = ' '))
        doc.append(i)

In [13]:
# Merge the past and current datasets

df1 = data[['text', 'label_update_with_scientific_impact', 'test']]
df2 = pd.DataFrame({'text': text, 'label_update_with_scientific_impact': [1]*len(text),
                    'test': ['no']*len(text)})

In [14]:
data = pd.concat([df1, df2])
data.index = range(len(data))

In [15]:
# keywords
cate1 = ['auswirken', 'Auswirkung', 'beeinflussen', 'beinflußen', 
 'Effekt', 'effektiv', 'Einfluss', 'Einfluß', 'Fortschritt', 'Impact', 'nachhaltig', 'nutzbar', 'Nutzbarmachung', 
 'Potential', 'Potenzial', 'umsetzen', 'Umsetzung', 'verändern', 'Veränderung', 'verbessern', 
 'Verbesserung', 'Verwertung', 'Verwertungsmöglichkeiten', 'wirksam', 'Wirksamkeit', 'Wirkung']
cate2 = ['beachtlich', 'Beitrag', 'beitragen', 'direkt', 'Einflussnahme', 'Einflußnahme', 'Einflußmöglichkeit', 
         'Einflussmöglichkeit', 'Einsatzmöglichkeiten', 'hochrelevant', 
         'Innovation', 'innovativ', 'realisierbar', 'realisieren', 'Realisierung', 'Ziel', 'zielführend']
cate3= 'abschätzbar, abschätzen, anwenden, Anwendung, Anwendungsfall, Anwendungsframework, Anwendungsszenario, Attraktivität, effizient, Entwicklung, Erfolg, Erfolgsaussichten, Ergebnisse, ermöglichen, erreichen, erzielen, Feedback, Frontend, Gewinn, gewinnen, gewinnorientiert, Hauptanwendungsfälle, indirekt, Infrastruktur, infrastrukturell, langfristig, lösen, Lösung, maßgeblich, messbar, meßbar, negativ, neu, nutzen, positiv, produktiv, Projektziele, reagieren, Reaktion, real-world, spürbar, strukturell, Überwindung, unmittelbar, Use_Case, Weiterentwicklung, Wertschöpfung, Wettbewerb, Wettbewerbsanalyse, Zukunft, zukünftig, Zweck'.split(', ')

In [16]:
# compute the frequencies of each keyword in texts as well as the aggregrated number
data['aaa'] = data['text'].apply(lambda x: len([word for word in x.split() if word in set(cate1)]))
data['bbb'] = data['text'].apply(lambda x: len([word for word in x.split() if word in set(cate2)]))
data['ccc'] = data['text'].apply(lambda x: len([word for word in x.split() if word in set(cate3)]))

In [17]:
data['total'] = data['aaa'] + data['bbb'] + data['ccc']
for i in set(cate1+cate2+cate3):
    data[str(i)] = data['text'].apply(lambda x: x.count(i))

/tmp/ipykernel_679427/2400321340.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[str(i)] = data['text'].apply(lambda x: x.count(i))
/tmp/ipykernel_679427/2400321340.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[str(i)] = data['text'].apply(lambda x: x.count(i))


In [18]:
# training - validation/testing set split
train = data[data['test'] == 'no']
test = data[data['test'] == 'yes']

In [19]:
len(train)

2054

In [20]:
len(test)

444

In [21]:
#shuffle data
train = train.sample(frac = 1, random_state = 10)

In [24]:
# best model parameters and result
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
iid = data.columns[3:]
clf = RandomForestClassifier(n_estimators = 1000, class_weight = 'balanced', min_samples_leaf = 6, max_depth=6).fit(train[iid], train['label_update_with_scientific_impact'])
final = clf.predict(test[iid])
print(classification_report(test['label_update_with_scientific_impact'], final))

              precision    recall  f1-score   support

           0       0.85      0.67      0.75       331
           1       0.41      0.66      0.51       113

    accuracy                           0.67       444
   macro avg       0.63      0.67      0.63       444
weighted avg       0.74      0.67      0.69       444



# Prediction

In [125]:
data = pd.read_csv('./ttparagraph_addmob.txt.gz')

### Using rule-based method to identify all irrelevant paragraphs

In [126]:

data['inhalt1'] = data['text'].str.contains('Inhaltsverzeichnis', case=True).apply(lambda x: 1 if x else 0)
data['inhalt2'] = data['text'].str.contains('Inhalt', case=True).apply(lambda x: 1 if x else 0)
inhalt1_sum = data.groupby('file').sum('inhalt1')
inhalt1_ind = inhalt1_sum[inhalt1_sum['inhalt1'] > 0].index.tolist()
inhalt2_ind = set(inhalt1_sum.index) - set(inhalt1_ind)

data['exclude_inhalt'] = False



list_exclude = []
for i in inhalt1_ind:
    data_part = data[data['file'] == i]
    ind_start = data_part.index.tolist()[0]
    data_part = data_part[data_part['inhalt1'] == True]
    if len(data_part) > 0:
        ind_end = data_part[data_part['inhalt1'] == True].index[0]
        range_inhalt = list(range(ind_start, ind_end+1))
        if len(range_inhalt) <= 30: # change
            list_exclude = list_exclude + range_inhalt
            
for i in inhalt2_ind:
    data_part = data[data['file'] == i]
    ind_start = data_part.index.tolist()[0]
    data_part = data_part[data_part['inhalt2'] == True]
    if len(data_part) > 0:
        ind_end = data_part[data_part['inhalt2'] == True].index[0]
        range_inhalt = list(range(ind_start, ind_end+1))
        if len(range_inhalt) <= 30: # change
            list_exclude = list_exclude + range_inhalt
            
            
data['exclude_inhalt'][list_exclude] = True

/tmp/ipykernel_656949/2894172796.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['exclude_inhalt'][list_exclude] = True


In [127]:
data['inhalt1'] = data['text'].str.contains('Bibliographie|Bibliografie|Literaturverzeichnis', case=True).apply(lambda x: 1 if x else 0)
data['inhalt2'] = data['text'].str.contains('Literatur', case=True).apply(lambda x: 1 if x else 0)
inhalt1_sum = data.groupby('file').sum('inhalt1')
inhalt1_ind = inhalt1_sum[inhalt1_sum['inhalt1'] > 0].index.tolist()
inhalt2_ind = set(inhalt1_sum.index) - set(inhalt1_ind)
data['exclude_inhalt1'] = False

In [128]:
list_exclude = []
for i in inhalt1_ind:
    data_part = data[data['file'] == i]
    ind_start = data_part.index.tolist()[-1]
    data_part = data_part[data_part['inhalt1'] == True]
    if len(data_part) > 0:
        ind_end = data_part.index[-1]
        range_inhalt = list(range(ind_end, ind_start+1))
        if len(range_inhalt) <= 75: # change
            list_exclude = list_exclude + range_inhalt
            
for i in inhalt2_ind:
    data_part = data[data['file'] == i]
    ind_start = data_part.index.tolist()[-1]
    data_part = data_part[data_part['inhalt2'] == True]
    if len(data_part) > 0:
        ind_end = data_part.index[-1]
        range_inhalt = list(range(ind_end, ind_start+1))
        if len(range_inhalt) <= 75: # change
            list_exclude = list_exclude + range_inhalt

In [129]:
data['exclude_inhalt1'][list_exclude] = True

/tmp/ipykernel_656949/2453373711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['exclude_inhalt1'][list_exclude] = True


In [130]:
data['text'] = data['text'].apply(lambda x: str(x))

In [131]:
data['num_token'] = data['text'].str.replace('.', '').str.replace(r"\s+", ' ').apply(lambda x: len(x.split(' ')))

/tmp/ipykernel_656949/2130238111.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['num_token'] = data['text'].str.replace('.', '').str.replace(r"\s+", ' ').apply(lambda x: len(x.split(' ')))
/tmp/ipykernel_656949/2130238111.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['num_token'] = data['text'].str.replace('.', '').str.replace(r"\s+", ' ').apply(lambda x: len(x.split(' ')))


In [132]:
data['exclude_num_token'] = (data['num_token'] < 25) # change

In [ ]:
# Calculate the occurences of keywords
cate1 = ['auswirken', 'Auswirkung', 'beeinflussen', 'beinflußen', 
 'Effekt', 'effektiv', 'Einfluss', 'Einfluß', 'Fortschritt', 'Impact', 'nachhaltig', 'nutzbar', 'Nutzbarmachung', 
 'Potential', 'Potenzial', 'umsetzen', 'Umsetzung', 'verändern', 'Veränderung', 'verbessern', 
 'Verbesserung', 'Verwertung', 'Verwertungsmöglichkeiten', 'wirksam', 'Wirksamkeit', 'Wirkung']
cate2 = ['beachtlich', 'Beitrag', 'beitragen', 'direkt', 'Einflussnahme', 'Einflußnahme', 'Einflußmöglichkeit', 
         'Einflussmöglichkeit', 'Einsatzmöglichkeiten', 'hochrelevant', 
         'Innovation', 'innovativ', 'realisierbar', 'realisieren', 'Realisierung', 'Ziel', 'zielführend']
cate3= 'abschätzbar, abschätzen, anwenden, Anwendung, Anwendungsfall, Anwendungsframework, Anwendungsszenario, Attraktivität, effizient, Entwicklung, Erfolg, Erfolgsaussichten, Ergebnisse, ermöglichen, erreichen, erzielen, Feedback, Frontend, Gewinn, gewinnen, gewinnorientiert, Hauptanwendungsfälle, indirekt, Infrastruktur, infrastrukturell, langfristig, lösen, Lösung, maßgeblich, messbar, meßbar, negativ, neu, nutzen, positiv, produktiv, Projektziele, reagieren, Reaktion, real-world, spürbar, strukturell, Überwindung, unmittelbar, Use_Case, Weiterentwicklung, Wertschöpfung, Wettbewerb, Wettbewerbsanalyse, Zukunft, zukünftig, Zweck'.split(', ')

In [ ]:
data['aaa'] = data['text'].apply(lambda x: len([word for word in x.split() if word in set(cate1)]))
data['bbb'] = data['text'].apply(lambda x: len([word for word in x.split() if word in set(cate2)]))
data['ccc'] = data['text'].apply(lambda x: len([word for word in x.split() if word in set(cate3)]))

In [ ]:
data['total'] = data['aaa'] + data['bbb'] + data['ccc']
for i in set(cate1+cate2+cate3):
    data[str(i)] = data['text'].apply(lambda x: x.count(i))

### Using the random to predict and concatenate rule-based and model prediction result

In [138]:
final1 = clf.predict(data[data.columns[12:]])

In [139]:
data['relevant'] = final1

In [140]:
aa = data[['text','Domain','Projects', 'file', 'exclude_inhalt', 'exclude_inhalt1', 'exclude_num_token', 'relevant']]

In [144]:
ind = aa[aa['exclude_inhalt'] == False][aa['exclude_inhalt1'] == False][aa['exclude_num_token'] == False][aa['relevant']==1].index

/tmp/ipykernel_656949/1710346095.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ind = aa[aa['exclude_inhalt'] == False][aa['exclude_inhalt1'] == False][aa['exclude_num_token'] == False][aa['relevant']==1].index


In [149]:
aa['relevant'] = 0

/tmp/ipykernel_656949/2946407759.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['relevant'] = 0


In [167]:
relevant = []
for i in range(len(aa)):
    if i in ind:
        relevant.append(1)
    else:
        relevant.append(0)

In [177]:
aa['relevant'] = relevant

/tmp/ipykernel_656949/1965354603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aa['relevant'] = relevant


In [182]:
# save results
aa[['text','Domain','Projects', 'file', 'relevant']].to_csv('model_result_extraction.csv')